## 3- Combine data from Inside Schools and Chalkbeat

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats.stats import pearsonr 

Load data from each website

In [2]:
df_is = pd.read_pickle("oct_3.pkl")
df_fund = pd.read_pickle("funding_per_student.pkl")

Set column names of Inside Schools dataframe

In [3]:
df_is = df_is[['How many students graduate in 4 years?','School Name','School Code','Address', 'Asian', 'Average daily attendance', 'Black',
       'English language learners', 'Enrollment',
       'Free or reduced priced lunch', 'Hispanic',
       'How many English language learners graduate in 4 years?',
       'How many graduates earn Advanced Regents diplomas?',
       'How many graduates stay enrolled in college for at least 3 semesters?',
       'How many students graduate with test scores high enough to enroll at CUNY without remedial help?',
       'How many students miss 18 or more days of school?',
       'How many students say they feel safe in the hallways, bathrooms and locker rooms?',
       'How many students take a college-level course or earn a professional certificate?',
       'How many students think bullying happens most or all of the time at this school?',
       'How many students were suspended?',
       'How many students with disabilities graduate in 4 years?',
       'How many teachers have 3 or more years of experience teaching?',
       'How many teachers say order and discipline are maintained at this school?',
       'How many teachers say the principal is an effective manager?',
       'How many teachers say they would recommend this school to other families?',
       'Metal detectors?', 'Other',  
       'Shared campus?', 'Students with disabilities', 'Teacher Effectiveness',
       'White', 'Years of principal experience at this school']]


Use close matches to change school names that differ slightly to match them between dataframes

In [13]:
import difflib

school_names=df_is['School Name'].map(lambda x: difflib.get_close_matches(x, df_fund['School'], n=1))
school_names = [s[0] if s else '' for s in school_names]
len(school_names)

433

Merge data frames to put inside school data and funding data together

In [74]:
df_merged = pd.merge(df_is,df_fund,left_on='New School Name',right_on='School',how='left')
#df_merged.head()

### Clean data for use in Linear Regression

Drop schools that don't have the target value available

In [75]:
df_merged = df_merged[pd.notnull(df_merged['How many students graduate in 4 years?'])]
#df_merged.info()

Drop data related to metal detectors, shared campuses, and students with disabilities due to lack of data (less than half of schools have this information)

In [76]:
df_merged=df_merged.drop(['Metal detectors?'], axis=1)

In [78]:
df_merged=df_merged.drop(['Shared campus?'], axis=1)

In [118]:
df_merged=df_merged.drop(['How many students with disabilities graduate in 4 years?'], axis=1)

Drop redundant School Name column

In [123]:
df_merged=df_merged.drop(['School'], axis=1)

Fill in missing values for a few schools that were missing information

In [80]:
df_merged['How many graduates stay enrolled in college for at least 3 semesters?'] = df_merged['How many graduates stay enrolled in college for at least 3 semesters?'].fillna(df_merged['How many students graduate with test scores high enough to enroll at CUNY without remedial help?'])

In [109]:
df_merged['How many students were suspended?'].fillna((df_merged['How many students were suspended?'].mean()), inplace=True)


Convert school funding column to float

In [136]:
df_merged['$ per Student'] = df_merged['$ per Student'].str.replace(',','')
df_merged['$ per Student'] = df_merged['$ per Student'].astype(float)
df_merged.head()

,How many students graduate in 4 years?,School Name,School Code,Address,Asian,Average daily attendance,Black,English language learners,Enrollment,Free or reduced priced lunch,...,How many teachers have 3 or more years of experience teaching?,How many teachers say order and discipline are maintained at this school?,How many teachers say the principal is an effective manager?,How many teachers say they would recommend this school to other families?,Other,Students with disabilities,Teacher Effectiveness,White,Years of principal experience at this school,$ per Student
1,78.0,Urban Assembly High School of Music and Art,13K350,49 Flatbush Avenue Extension Brooklyn NY 11201,1.0,81.0,77.0,3.0,221.0,86.0,...,47.0,55.0,73.0,73.0,1.0,23.0,2,2.0,12.0,25572.0
2,96.0,Queens Gateway to Health Sciences Secondary Sc...,28Q680,160-20 Goethals Avenue Jamaica NY 11432,51.0,98.0,27.0,1.0,690.0,63.0,...,89.0,65.0,53.0,68.0,8.0,12.0,4,4.0,5.2,19613.0
3,70.0,Women's Academy of Excellence,08X282,456 White Plains Road Bronx NY 10473,11.0,75.0,41.0,21.0,281.0,93.0,...,87.0,92.0,83.0,88.0,4.0,19.0,3,12.0,13.0,24451.0
4,97.0,World Journalism Preparatory: A College Board ...,25Q285,34-65 192nd Street Flushing NY 11358,15.0,95.0,4.0,2.0,567.0,56.0,...,76.0,97.0,100.0,95.0,2.0,19.0,4,44.0,0.6,20175.0
5,74.0,ACORN Community High School,13K499,561 Grand Avenue Brooklyn NY 11238,2.0,80.0,79.0,5.0,193.0,82.0,...,100.0,67.0,67.0,67.0,1.0,28.0,3,3.0,7.0,31684.0


Save clean dataframe for use in Linear Regression

In [137]:
df_merged.to_pickle("clean_school_df.pkl")